In [ ]:
%pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving test_dataset.json to test_dataset.json
Saving train_dataset.json to train_dataset.json
Saving val_dataset.json to val_dataset.json


In [ ]:
import json
from datasets import Dataset, DatasetDict

def load_dataset(path):
    with open(path) as f:
        data = json.load(f)
    return Dataset.from_list([{"input_text": d["input"], "target_text": d["output"]} for d in data])

def get_dataset():
    train_data = load_dataset("train_dataset.json")
    val_data = load_dataset("val_dataset.json")
    test_data = load_dataset("test_dataset.json")  # note the typo, may be 'test_dataset.json'

    print("✅ Loaded datasets:")
    print(f"Train size: {len(train_data)}, Val size: {len(val_data)}, Test size: {len(test_data)}")

    return DatasetDict({
        "train": train_data,
        "validation": val_data,
        "test": test_data
    })


In [ ]:
from transformers import T5Tokenizer

def tokenize_data(dataset, tokenizer, max_input_len=256, max_target_len=512):
    def preprocess(example):
        input_enc = tokenizer(example["input_text"], truncation=True, padding="max_length", max_length=max_input_len)
        target_enc = tokenizer(example["target_text"], truncation=True, padding="max_length", max_length=max_target_len)
        return {
            "input_ids": input_enc["input_ids"],
            "attention_mask": input_enc["attention_mask"],
            "labels": target_enc["input_ids"]
        }
    tokenized = dataset.map(preprocess, batched=True)
    print("✅ Tokenization complete")
    return tokenized


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments

def fine_tune(tokenized_datasets):
    # ✅ Use google/flan-t5-base without quantization
    model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

    training_args = TrainingArguments(
        output_dir="./results",
        eval_strategy="steps",
        eval_steps=100,
        logging_steps=50,
        save_steps=200,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=3,
        save_total_limit=2,
        fp16=False,                # Set to True if your GPU supports it
        logging_dir="./logs",
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"]
    )

    print("🚀 Starting training...")
    trainer.train()
    print("✅ Training complete")
    model.save_pretrained("flan-t5-custom-finetuned")
    return model


In [ ]:
import torch

In [ ]:
import re
import torch

def fix_xml(prediction: str) -> str:
    # Add < if missing at the start of tags
    prediction = re.sub(r'(?<!<)(\w+)(\s[^<>]*?/?>)', r'<\1\2', prediction)
    # Add > if missing at the end
    prediction = re.sub(r'(/?>)(?!>)', r'\1>', prediction)
    return prediction

def predict(model, tokenizer, dataset, max_input_len=256, max_target_len=512):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    for example in dataset.select(range(3)):
        inputs = tokenizer(example["input_text"], return_tensors="pt", truncation=True, padding=True, max_length=max_input_len)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        output_ids = model.generate(**inputs, max_length=max_target_len)
        prediction = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        fixed_prediction = fix_xml(prediction)

        print("📥 Input:", example["input_text"])
        print("✅ Raw Prediction:", prediction)
        print("🛠 Fixed Prediction:", fixed_prediction)
        print("🎯 Ground Truth:", example["target_text"])
        print("-" * 50)


In [ ]:
from transformers import T5Tokenizer

if __name__ == "__main__":
    tokenizer = T5Tokenizer.from_pretrained("t5-base")

    # Load and preprocess
    datasets = get_dataset()
    tokenized_datasets = tokenize_data(datasets, tokenizer)

    # Train
    model = fine_tune(tokenized_datasets)

    # Predict
    print("🔍 Running prediction on test set...")
    predict(model, tokenizer, datasets["test"])


✅ Loaded datasets:
Train size: 1600, Val size: 200, Test size: 200


Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

✅ Tokenization complete
🚀 Starting training...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
100,2.089400,0.507768
200,0.176700,0.066588
300,0.079400,0.036036
400,0.055500,0.028956
500,0.042400,0.025838
600,0.040500,0.024559
700,0.037400,0.024278
800,0.036600,0.023819
900,0.033100,0.023384
1000,0.031300,0.023290


✅ Training complete
🔍 Running prediction on test set...
📥 Input: Create course offering ART401 ART Course with Dr. Williams on Friday from 9:00 AM to 10:00 AM in room THTR202 with limit 20 students
✅ Prediction: offerings campus="MAIN" year="2024" term="Fall">offering id="336" offered="true">course id="490" subject="ART" courseNbr="401" title="ART Course">class id="33695" suffix="1" type="LEC" limit="20">time days="F" startTime="0900" endTime="1000"/>room building="THTR" roomNbr="202"/>instructor id="67" fname="Dr." lname="Williams" lead="true"/>/class>/course>/offering>/offerings>
🎯 Ground Truth: <offerings campus="MAIN" year="2024" term="Fall"><offering id="5858" offered="true"><course id="999" subject="ART" courseNbr="401" title="ART Course"><class id="92516" suffix="1" type="SEM" limit="20"><time days="F" startTime="0900" endTime="1000"/><room building="THTR" roomNbr="202"/><instructor id="79" fname="Dr." lname="Williams" lead="true"/></class></course></offering></offerings>
------